In [1]:
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType

import datasets
import wandb

from tqdm import tqdm

/home/vlialin/miniconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
For effortless bug reporting copy-paste your error into this form: https://docs.google.com/forms/d/e/1FAIpQLScPB8emS3Thkp66nvqwmjTEgxp8Y9ufuWTzFyr9kJ5AoI47dQ/viewform?usp=sf_link
CUDA SETUP: CUDA runtime path found: /home/vlialin/miniconda3/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/vlialin/miniconda3/lib/python3.7/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


In [2]:
BATCH_SIZE = 24
MAX_LENGTH = 128

data = datasets.load_dataset("c4", "en", split="train", streaming=True)
data = data.shuffle(seed=42)

tokenizer = AutoTokenizer.from_pretrained("t5-base")

def preprocess_batched(batch):
    batch = tokenizer(
        batch["text"],
        max_length=MAX_LENGTH,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )
    return batch

data_mapped = data.map(preprocess_batched, batched=True, batch_size=1000, remove_columns=["text", "timestamp", "url"])

def collate_fn(batch_list):
    batch = {
        "input_ids": torch.stack([example["input_ids"] for example in batch_list]),
        "attention_mask": torch.stack([example["attention_mask"] for example in batch_list]),
    }
    return batch

def batch_fn(dataset, batch_size):
    batch = []
    for example in dataset:
        batch.append(example)
        if len(batch) == batch_size:
            batch = collate_fn(batch)
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch

data_mapped.batch = lambda batch_size: batch_fn(data_mapped, batch_size)

/home/vlialin/miniconda3/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [3]:
USE_PEFT = True
TRAIN_LN = True
NUM_TRAINING_STEPS = 10_000

device = "cuda:1"

model_config = AutoConfig.from_pretrained("gpt2-large")
model = AutoModelForCausalLM.from_config(model_config)

if USE_PEFT:
    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.1,
    )

    model = get_peft_model(peft_config, model)

    for name, param in model.named_parameters():
        if TRAIN_LN and "ln_" in name:
            param.requires_grad = True
        if "lm_head" in name:
            param.requires_grad = True
        if "transformer.wte" in name:
            param.requires_grad = True
        if "transformer.wpe" in name:
            param.requires_grad = True

    model.print_trainable_parameters()

model = model.to(device)

n_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
n_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
p_trainable_params = n_trainable_params / n_total_params

trainable_params = (p for p in model.parameters() if p.requires_grad)
trainable_params_names = [name for name, p in model.named_parameters() if p.requires_grad]

optimizer = torch.optim.Adam(trainable_params, lr=1e-4)
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=1_000, num_training_steps=NUM_TRAINING_STEPS)

_config = {
    "using_peft": USE_PEFT,
    "layer_norm_trainable": TRAIN_LN,
    "peft_config": peft_config.to_dict(),
    "total_params": n_total_params,
    "trainable_params": n_trainable_params,
    "percent_trainable_params": p_trainable_params,
    "name_trainable_params": trainable_params_names,
    "dataset": "c4",
    "batch_size": BATCH_SIZE,
    "max_length": MAX_LENGTH,
    "model": model_config.to_dict(),
    "scheduler": "linear",
    "device": str(device),
}

wandb.init(project="peft_pretraining", config=_config)
pbar = tqdm(total=NUM_TRAINING_STEPS)

trainable params: 67114240 || all params: 775504640 || trainable%: 8.65426672366525


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: guitaricet. Use `wandb login --relogin` to force relogin


  0%|          | 0/10000 [00:00<?, ?it/s]

In [4]:
model.base_model.transformer.wte.weight.requires_grad

True

In [5]:
for epoch in range(1):
    data_mapped.set_epoch(epoch)
    for batch in data_mapped.batch(batch_size=BATCH_SIZE):
        pbar.update(1)
        optimizer.zero_grad()

        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch["input_ids"].clone()
        labels[labels == 0] = -100

        loss = model(**batch, labels=labels).loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        lr = scheduler.get_last_lr()[0]
        wandb.log({
            "loss": loss.item(),
            "lr": lr,
        })

  6%|▌         | 561/10000 [07:51<2:08:33,  1.22it/s]